In [1]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from typing import List, Optional, Tuple, Dict
import sys
from functools import partial
from tqdm import tqdm
import json
import seaborn as sns
import pandas as pd
import multiprocessing
import pickle

from request_patching import request_patch_one_pair, create_patch_request_dict

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device =', device)
torch.set_grad_enabled(False)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

/home/jsalle/.conda/envs/athena_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = cpu
CUDA is not available. Listing CPUs instead.
48


In [2]:
dict = create_patch_request_dict('pythia-1b',
                                 dataset='dialogs')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 3


In [ ]:
with open(f'dash_app/patch_request_dictionaries/pythia-1b_dialogs.pkl', 'rb') as f:
    patch_request_dict = pickle.load(f)

In [ ]:
for i, key in enumerate(patch_request_dict):
    R2_C2 = patch_request_dict[f'pair_{i}']['patching_result']
    print(R2_C2)

['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'happy']
['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'happy']
['b', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'fr', 'fr', 'em']


### temp

In [ ]:
dict = create_patch_request_dict('gpt2-small',
                                 dataset='dialogs')

In [ ]:
dict = create_patch_request_dict('pythia-2.8b',
                                 dataset='dialogs')